In [9]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard.writer import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [15]:
%run -i data_generation/simple_linear_regression.py
%run -i data_preparetion/v0.py
%run -i model_configuration/v0.py

device:cpu


In [16]:
def make_train_step(model, loss_fn, optimizer):
    # 构建在训练循环中执行一个步骤的函数
    def perform_train_step(x, y):
        # 设置模型为训练模式
        model.train()

        # 第1步：计算模型的预测输出--前向传递
        yhat = model(x)
        # 第2步：计算损失
        loss = loss_fn(yhat, y)
        # 第3步：计算参数a和b的梯度
        loss.backward()
        # 第4步：使用梯度和学习率更新参数
        optimizer.step()
        optimizer.zero_grad()

        # 返回损失
        return loss.item()
    
    # 返回将在训练循环内调用的函数
    return perform_train_step

In [17]:
%run -i data_preparetion/v0.py

device:cpu


In [18]:
%%writefile model_configuration/v1.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 设置学习率
lr = 0.1

torch.manual_seed(42)
# 现在可以创建一个模型并立即将其发送到设备
model = nn.Sequential(nn.Linear(1,1)).to(device)

# 定义SGD优化器来更新参数
optimizer = optim.SGD(model.parameters(), lr=lr)

# 定义MSE损伤函数
loss_fn = nn.MSELoss(reduction='mean')

# 为模型、损伤函数和优化器创建train_step函数
train_step = make_train_step(model, loss_fn, optimizer)

Writing model_configuration/v1.py


In [19]:
%run -i model_configuration/v1.py

In [20]:
train_step

<function __main__.make_train_step.<locals>.perform_train_step(x, y)>

In [21]:
%%writefile model_training/v1.py

# 定义周期数
n_epochs = 1000

losses = []

# 对于每个周期...
for epoch in range(n_epochs):
    # 执行一个训练步骤并返回相应的损失
    loss = train_step(x_train_tensor, y_train_tensor)
    losses.append(loss)

Writing model_training/v1.py


In [22]:
%run -i model_training/v1.py

In [23]:
losses

[0.7255967855453491,
 0.4431508183479309,
 0.28266122937202454,
 0.19111186265945435,
 0.13854403793811798,
 0.10802850872278214,
 0.0899982899427414,
 0.07904667407274246,
 0.0721178874373436,
 0.06748442351818085,
 0.0641695037484169,
 0.061620552092790604,
 0.059524692595005035,
 0.057704489678144455,
 0.056059159338474274,
 0.05453111603856087,
 0.05308728665113449,
 0.051708467304706573,
 0.050383228808641434,
 0.04910467937588692,
 0.04786836355924606,
 0.046671345829963684,
 0.04551146179437637,
 0.044387079775333405,
 0.04329680651426315,
 0.04223943501710892,
 0.04121391102671623,
 0.04021921381354332,
 0.03925437480211258,
 0.03831850737333298,
 0.03741069883108139,
 0.03653012588620186,
 0.035675954073667526,
 0.03484740108251572,
 0.034043699502944946,
 0.03326408565044403,
 0.032507866621017456,
 0.031774312257766724,
 0.031062761321663857,
 0.03037254884839058,
 0.029703030362725258,
 0.02905358001589775,
 0.0284236129373312,
 0.027812525629997253,
 0.02721976675093174,
 

In [24]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]])), ('0.bias', tensor([1.0235]))])


In [25]:
class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        super().__init__()
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)
    
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))


In [26]:
train_data = TensorDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))


In [27]:
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

In [28]:
next(iter(train_loader))

[tensor([[0.2809],
         [0.3253],
         [0.1560],
         [0.5924],
         [0.0651],
         [0.8872],
         [0.4938],
         [0.0055],
         [0.1409],
         [0.0885],
         [0.1849],
         [0.7290],
         [0.8662],
         [0.3117],
         [0.6842],
         [0.1987]]),
 tensor([[1.5846],
         [1.8057],
         [1.2901],
         [2.1687],
         [1.1559],
         [2.8708],
         [1.9060],
         [1.0632],
         [1.1211],
         [1.0708],
         [1.5888],
         [2.4927],
         [2.6805],
         [1.7637],
         [2.3492],
         [1.2654]])]

In [29]:
%%writefile data_preparetion/v1.py

# 数据在numpy数组中
# 但需要将它们转化为PyTorch的张量
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

# 构建Dataset
train_data = TensorDataset(x_train_tensor, y_train_tensor)

# 构建DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

Writing data_preparetion/v1.py


In [30]:
%run -i data_preparetion/v1.py

In [31]:
%%writefile model_training/v2.py

# 定义周期数
n_epochs = 1000

losses = []

for epoch in range(n_epochs):
    # 内循环
    mini_batch_losses = []
    for x_batch, y_batch in train_loader:
        # 数据集存在于CPU中，小批量也是如此
        # 因此需要将这些小批量，发送到模型存在的设备
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # 执行一个训练步骤
        # 并返回小批量的相应损失
        mini_batch_loss = train_step(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)

    # 计算所有小批量的平均损失-- 这是周期损失
    loss = np.mean(mini_batch_losses)

    losses.append(loss)

Writing model_training/v2.py


In [32]:
%run -i model_training/v2.py

In [33]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9698]])), ('0.bias', tensor([1.0246]))])


In [34]:
%run -i data_preparetion/v1.py
%run -i model_configuration/v1.py
%run -i model_training/v2.py

In [35]:
def mini_batch(device, data_loader, step):
    mini_batch_losses = []
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = step(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)

    loss = np.mean(mini_batch_losses)
    return loss

In [36]:
%%writefile model_training/v3.py

# 定义周期数
n_epochs = 200

losses = []

for epoch in range(n_epochs):
    loss = mini_batch(device, train_loader, train_step)
    losses.append(loss)

Writing model_training/v3.py


In [37]:
%run -i data_preparetion/v1.py
%run -i model_configuration/v1.py
%run -i model_training/v3.py

In [38]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9684]])), ('0.bias', tensor([1.0219]))])


In [39]:
%%writefile data_preparetion/v2.py

torch.manual_seed(13)

# 在拆分之前从numpy数组构建张量
x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

# 构建包含所有数据点的数据集
dataset = TensorDataset(x_tensor, y_tensor)

# 执行拆分
ratio = .8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train

train_data, val_data = random_split(dataset, [n_train, n_val])

# 构建每个集合的加载器
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=16)

Writing data_preparetion/v2.py


In [40]:
%run -i data_preparetion/v2.py

In [41]:
def make_val_step(model, loss_fn):
    # 在验证循环中构建执行步骤的函数
    def perform_val_step(x, y):
        # 设置模型为评估模式
        model.eval()

        # 第1步：计算模型的预测输出--前向传递
        yhat = model(x)
        # 第2步：计算损失
        loss = loss_fn(yhat, y)
        # 因为评估期间不更新参数
        # 所以无需技术第3步和第4步
        return loss.item()
    
    return perform_val_step

In [42]:
%%writefile model_configuration/v2.py

device = 'cude' if torch.cuda.is_available() else 'cpu'

# 设置学习率
lr = 0.1

torch.manual_seed(42)
# 现在可以创建模型并立即发送到设备
model = nn.Sequential(nn.Linear(1, 1)).to(device)

# 定义GSD优化器来更新
optimizer = optim.SGD(model.parameters(), lr=lr)

# 定义MSE损失函数
loss_fn = nn.MSELoss(reduction='mean')

# 为模型、损伤函和优化器创建train_step函数
train_step = make_train_step(model, loss_fn, optimizer)

# 为模型和损失函数创建val_step函数
val_step = make_val_step(model, loss_fn)

Writing model_configuration/v2.py


In [43]:
%run -i model_configuration/v2.py

In [44]:
%%writefile model_training/v4.py

# 定义周期数
n_epochs = 200

losses = []
val_losses = []

for epoch in range(n_epochs):
    # 内循环
    loss = mini_batch(device, train_loader, train_step)
    losses.append(loss)

    # 验证--验证中没有梯度
    with torch.no_grad():
        val_loss = mini_batch(device, val_loader, val_step)
        val_losses.append(val_loss)

Writing model_training/v4.py


In [45]:
%run -i model_training/v4.py

In [46]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9438]])), ('0.bias', tensor([1.0287]))])


In [47]:
%run -i data_preparetion/v2.py
%run -i model_configuration/v2.py
%run -i model_training/v4.py

In [48]:
# Load the Tensorboard notebook extension
%load_ext tensorboard

In [51]:
writer = SummaryWriter('runs/test')

In [53]:
# 获取特征(dummy_x)和标签(dummy_y)的元组
dummy_x, dummy_y = next(iter(train_loader))

# 由于模型已发送到设备，因此需要对数据执行相同的操作
# 即使在这里，模型和数据也需要再同一台设备上
writer.add_graph(model, dummy_x.to(device))

In [54]:
writer.add_scalars(
    main_tag="loss",
    tag_scalar_dict = {"training":loss, 'validation': val_loss},
    global_step=epoch
)

In [55]:
%run -i data_preparetion/v2.py

In [61]:
%%writefile model_configuration/v3.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 设置学习率
lr = 0.1

torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1, 1)).to(device)

optimizer = optim.SGD(model.parameters(), lr=lr)

loss_fn = nn.MSELoss(reduction='mean')

train_step_fn = make_train_step(model, loss_fn, optimizer)

val_step_fn = make_val_step(model, loss_fn)

writer = SummaryWriter('runs/simple_linear_regression')

# 获取单个小样本，一遍可以使用add_graph
x_sample, y_sample = next(iter(train_loader))
writer.add_graph(model, x_sample.to(device))

Overwriting model_configuration/v3.py


In [62]:
%run -i model_configuration/v3.py

In [63]:
%%writefile model_training/v5.py

# 定义周期数
n_epochs = 200

losses = []
val_losses = []

for epoch in range(n_epochs):
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

    # 验证
    with torch.no_grad():
        val_loss = mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)

    # 在主标签“损失”下记录每个周期的损失
    writer.add_scalars(
        main_tag='loss',
        tag_scalar_dict={'training':loss, 'validation':val_loss},
        global_step = epoch
    )

# 关闭编写器
writer.close()

Overwriting model_training/v5.py


In [66]:
%run -i model_training/v5.py

In [65]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9432]])), ('0.bias', tensor([1.0263]))])


In [67]:
# 保存模型
checkpoint = {
    'epoch': n_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
    'val_loss': val_loss,
}
torch.save(checkpoint, 'model_checkpoint.pth')

In [68]:
%run -i data_preparetion/v2.py
%run -i model_configuration/v3.py

In [69]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[0.7645]])), ('0.bias', tensor([0.8300]))])


In [70]:
checkpoint = torch.load('model_checkpoint.pth')

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
saved_epoch = checkpoint['epoch']
save_losses = checkpoint['loss']
save_val_losses = checkpoint['val_loss']

model.train()

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
)

In [71]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9411]])), ('0.bias', tensor([1.0230]))])


In [72]:
%run -i model_training/v5.py

In [73]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9432]])), ('0.bias', tensor([1.0263]))])
